In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

def fix_zip(s):
    if "-" in str(s):
        temp = str(s)
        temp = temp[0:temp.index("-")]
        return float(temp)
    return float(s)

In [ ]:
popdf = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQPc3h46MERwOc1tBe14YMvbdIU7OUb718QvPLTijNwb3LQxVHEoI2p_2agpe9Qdxhb0lKNakK_bCpn/pub?gid=699144644&single=true&output=csv")
popdf = popdf.loc[:,["zip_code","y-2016"]]
popdf.columns = ["Zipcode", "Population"]
popdf

,Zipcode,Population
0,601,17800
1,602,39716
2,603,51565
3,606,6320
4,610,27976
...,...,...
33115,99923,0
33116,99925,908
33117,99926,1667
33118,99927,81


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/gooddata.csv")
df["Zipcode"] = df["Zipcode"].apply(fix_zip)
df = df.join(popdf.set_index("Zipcode"), on="Zipcode")
df

,Unnamed: 0,Start_Lat,Start_Lng,Street,City,County,Zipcode,Weather_Timestamp,Weather_Condition,Population
0,0,39.865147,-84.058723,I-70 E,Dayton,Montgomery,45424.0,2016-02-08 05:58:00,Light Rain,50670.0
1,1,39.928059,-82.831184,Brice Rd,Reynoldsburg,Franklin,43068.0,2016-02-08 05:51:00,Light Rain,54128.0
2,2,39.063148,-84.032608,State Route 32,Williamsburg,Clermont,45176.0,2016-02-08 06:56:00,Overcast,9066.0
3,3,39.747753,-84.205582,I-75 S,Dayton,Montgomery,45417.0,2016-02-08 07:38:00,Mostly Cloudy,30606.0
4,4,39.627781,-84.188354,Miamisburg Centerville Rd,Dayton,Montgomery,45459.0,2016-02-08 07:53:00,Mostly Cloudy,27897.0
...,...,...,...,...,...,...,...,...,...,...
3513612,3513612,34.002480,-117.379360,Pomona Fwy E,Riverside,Riverside,92501.0,2019-08-23 17:53:00,Fair,21975.0
3513613,3513613,32.766960,-117.148060,I-8 W,San Diego,San Diego,92108.0,2019-08-23 18:53:00,Fair,20899.0
3513614,3513614,33.775450,-117.847790,Garden Grove Fwy,Orange,Orange,92866.0,2019-08-23 18:53:00,Partly Cloudy,15760.0
3513615,3513615,33.992460,-118.403020,San Diego Fwy S,Culver City,Los Angeles,90230.0,2019-08-23 18:51:00,Fair,32692.0


In [ ]:
streetdata = df.loc[:,["Zipcode","Street"]].groupby(["Zipcode", "Street"]).agg({"Street": "count"})
streetdata

Street
Zipcode Street                    
1001.0  Agawam Rotary            5
        MA-57 E                  2
        MA-57 W                  1
        Maple St                 1
        Suffield St              3
...                            ...
99371.0  E Highway 26            1
         S Highway 261           1
        E Highway 26             1
99401.0  State Highway 129       5
99403.0 Greenbelt Trl            1

[356885 rows x 1 columns]

In [ ]:
formatted_pop = df.loc[:,["Zipcode","Street","Population"]].groupby(["Zipcode", "Street"]).agg({"Population": "mean"})
formatted_pop

Population
Zipcode Street                        
1001.0  Agawam Rotary          17423.0
        MA-57 E                17423.0
        MA-57 W                17423.0
        Maple St               17423.0
        Suffield St            17423.0
...                                ...
99371.0  E Highway 26            414.0
         S Highway 261           414.0
        E Highway 26             414.0
99401.0  State Highway 129       264.0
99403.0 Greenbelt Trl          20080.0

[356885 rows x 1 columns]

In [ ]:
streetdata["Population"] = formatted_pop["Population"]
streetdata["Score"] = streetdata["Street"]/streetdata["Population"]
streetdata

Street  Population     Score
Zipcode Street                                          
1001.0  Agawam Rotary            5     17423.0  0.000287
        MA-57 E                  2     17423.0  0.000115
        MA-57 W                  1     17423.0  0.000057
        Maple St                 1     17423.0  0.000057
        Suffield St              3     17423.0  0.000172
...                            ...         ...       ...
99371.0  E Highway 26            1       414.0  0.002415
         S Highway 261           1       414.0  0.002415
        E Highway 26             1       414.0  0.002415
99401.0  State Highway 129       5       264.0  0.018939
99403.0 Greenbelt Trl            1     20080.0  0.000050

[356885 rows x 3 columns]

In [ ]:
streetdata["Score"] = streetdata["Score"].map(lambda s: 100 if s >= 0.0005 else s)
streetdata = streetdata[streetdata.Score != 100]
streetdata.describe()

,Street,Population,Score
count,283366.000000,282641.000000,282641.000000
mean,2.643076,30055.876065,0.000116
std,3.599232,18316.712945,0.000112
min,1.000000,2001.000000,0.000009
25%,1.000000,16110.000000,0.000035
50%,1.000000,28103.000000,0.000069
75%,3.000000,40477.000000,0.000158
max,434.000000,115104.000000,0.000500


Use this to get mean, std to rank streets^^^

In [ ]:
numscores = streetdata["Score"].value_counts()
index_arr = numscores.index.to_numpy()
val_arr = numscores.to_numpy()
#result = []
#for i, j in zip(index_arr,val_arr):
  #result.append([i,j])
#result
output = pd.DataFrame({"score": index_arr, "num": val_arr})
output = output.set_index("score").sort_index()
#output.to_csv("scores.csv")

In [ ]:
output.describe()

,num
count,49340.000000
mean,5.728435
std,9.016296
min,1.000000
25%,1.000000
50%,2.000000
75%,6.000000
max,155.000000


Output tables 
Accidents: lat, long, time, street, zip

Scores:
Street, Zip, Score

In [ ]:
accidents = df.loc[:, ["Start_Lat","Start_Lng","Weather_Timestamp","Street","Zipcode"]]
accidents.columns = ["lat","long","time","street","zip"]
accidents

,lat,long,time,street,zip
0,39.865147,-84.058723,2016-02-08 05:58:00,I-70 E,45424.0
1,39.928059,-82.831184,2016-02-08 05:51:00,Brice Rd,43068.0
2,39.063148,-84.032608,2016-02-08 06:56:00,State Route 32,45176.0
3,39.747753,-84.205582,2016-02-08 07:38:00,I-75 S,45417.0
4,39.627781,-84.188354,2016-02-08 07:53:00,Miamisburg Centerville Rd,45459.0
...,...,...,...,...,...
3513612,34.002480,-117.379360,2019-08-23 17:53:00,Pomona Fwy E,92501.0
3513613,32.766960,-117.148060,2019-08-23 18:53:00,I-8 W,92108.0
3513614,33.775450,-117.847790,2019-08-23 18:53:00,Garden Grove Fwy,92866.0
3513615,33.992460,-118.403020,2019-08-23 18:51:00,San Diego Fwy S,90230.0


In [ ]:
scores_index = streetdata.index.to_numpy()
zipcodes = []
streetnames = []
for i in scores_index:
  zipcodes.append(i[0])
  streetnames.append(i[1])
scores = pd.DataFrame({"street": streetnames, "zip": zipcodes})
scores["score"] = streetdata["Score"].to_numpy()
scores

,street,zip,score
0,Agawam Rotary,1001.0,0.000287
1,MA-57 E,1001.0,0.000115
2,MA-57 W,1001.0,0.000057
3,Maple St,1001.0,0.000057
4,Suffield St,1001.0,0.000172
...,...,...,...
283361,N 2nd Ave,99362.0,0.000048
283362,S 12th Ave,99362.0,0.000024
283363,S Highway 125,99362.0,0.000072
283364,US Highway 12,99362.0,0.000024


In [ ]:
accidents.to_csv("accidentsTable.csv")
scores.to_csv("scoresTable.csv")